In [1]:
import pandas as pd
import fasttext
import time
import numpy as np
import multiprocessing as mp
from multiprocessing import Pool
import csv
import fastparquet
import spacy
import glob, os
import re
from os.path import isfile
import seaborn as sns

In [2]:
fmodel = fasttext.load_model('/data/dharp/packages/lid.176.bin')
nlp = spacy.load('en_core_web_lg')
#nlp = spacy.load('en_core_web_trf')

In [3]:
keep_string=r"(.+_(NOUN|ADV|VERB|ADJ|X|PRT|CONJ|PRON|DET|ADP|NUM|\.)|_END_)\s*"
large_files=['a_','an','of','to','in','ad','wh','be','ha','is','co','wa','he','no','it','wi','fo','re','as','on','we','punctuation','th','ma','pr','ar','ip','sh','ca','so','hi','bu','al','se','de','by','wo','st','fr','di','mo','su','at','or','yo','me','li','pa','do','ex','le','pe','po','if','ne','fi','un','fa','sa','ch','la','lo','ac','ho','mu','go','si','en','ev','tr']
nn='(?!(?:NOUN|PROPN)).*'
comp='(?:ADJ|NOUN|PROPN)\s(?:NOUN|PROPN)'
word='.*'
ner_cats=['CARDINAL', 'DATE', 'EVENT', 'FAC', 'GPE', 'LANGUAGE', 'LAW', 'LOC', 'MONEY', 'NORP', 'ORDINAL', 'ORG', 'PERCENT', 'PERSON', 'PRODUCT', 'QUANTITY', 'TIME', 'WORK_OF_ART']
n1=f'^{comp}\s{nn}\s{comp}$'
n2=f'^{comp}\s{nn}\s{word}\s{word}$'
n3=f'^{nn}\s{comp}\s{nn}\s{word}$'
n4=f'^{word}\s{nn}\s{comp}\s{nn}$'
n5=f'^{word}\s{word}\s{nn}\s{comp}$'

In [4]:
done_files=[]
cur_dir='/data/dharp/compounds/datasets/google/'
for file in glob.glob(f"{cur_dir}*.pkl"):
    done_files.append(file.split('.')[0].split('/')[-1][:2])

done_bigrams=list(set(done_files))
fivegram_list='0 1 2 3 4 5 6 7 8 9 a_ aa ab ac ad ae af ag ah ai aj ak al am an ao ap aq ar as at au av aw ax ay az b_ ba bb bc bd be bf bg bh bi bj bk bl bm bn bo bp bq br bs bt bu bv bw bx by bz c_ ca cb cc cd ce cf cg ch ci cj ck cl cm cn co cp cq cr cs ct cu cv cw cx cy cz d_ da db dc dd de df dg dh di dj dk dl dm dn do dp dq dr ds dt du dv dw dx dy dz e_ ea eb ec ed ee ef eg eh ei ej ek el em en eo ep eq er es et eu ev ew ex ey ez f_ fa fb fc fd fe ff fg fh fi fj fk fl fm fn fo fp fq fr fs ft fu fv fw fx fy fz g_ ga gb gc gd ge gf gg gh gi gj gk gl gm gn go gp gq gr gs gt gu gv gw gx gy gz h_ ha hb hc hd he hf hg hh hi hj hk hl hm hn ho hp hq hr hs ht hu hv hw hx hy hz i_ ia ib ic id ie if ig ih ii ij ik il im in io ip iq ir is it iu iv iw ix iy iz j_ ja jb jc jd je jf jg jh ji jj jk jl jm jn jo jp jq jr js jt ju jv jw jx jy jz k_ ka kb kc kd ke kf kg kh ki kj kk kl km kn ko kp kq kr ks kt ku kv kw kx ky kz l_ la lb lc ld le lf lg lh li lj lk ll lm ln lo lp lq lr ls lt lu lv lw lx ly lz m_ ma mb mc md me mf mg mh mi mj mk ml mm mn mo mp mq mr ms mt mu mv mw mx my mz n_ na nb nc nd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt nu nv nw nx ny nz o_ oa ob oc od oe of og oh oi oj ok ol om on oo op oq or os ot other ou ov ow ox oy oz p_ pa pb pc pd pe pf pg ph pi pj pk pl pm pn po pp pq pr ps pt pu punctuation pv pw px py pz q_ qa qb qc qd qe qf qg qh qi qj ql qm qn qo qp qq qr qs qt qu qv qw qx qy qz r_ ra rb rc rd re rf rg rh ri rj rk rl rm rn ro rp rq rr rs rt ru rv rw rx ry rz s_ sa sb sc sd se sf sg sh si sj sk sl sm sn so sp sq sr ss st su sv sw sx sy sz t_ ta tb tc td te tf tg th ti tj tk tl tm tn to tp tq tr ts tt tu tv tw tx ty tz u_ ua ub uc ud ue uf ug uh ui uj uk ul um un uo up uq ur us ut uu uv uw ux uy uz v_ va vb vc vd ve vf vg vh vi vj vk vl vm vn vo vp vq vr vs vt vu vv vw vx vy vz w_ wa wb wc wd we wf wg wh wi wj wk wl wm wn wo wp wq wr ws wt wu wv ww wx wy wz x_ xa xb xc xd xe xf xg xh xi xj xk xl xm xn xo xp xq xr xs xt xu xv xw xx xy xz y_ ya yb yc yd ye yf yg yh yi yj yk yl ym yn yo yp yq yr ys yt yu yv yw yx yy yz z_ za zb zc zd ze zf zg zh zi zj zk zl zm zn zo zp zq zr zs zt zu zv zw zx zy zz'
fivegram_list=fivegram_list.split(" ")
to_do_list=set(fivegram_list).difference(done_bigrams)
done_list='0 1 2 3 4 5 6 7 8 9 other punctuation'
done_list=done_list.split(" ")
to_do_list=list(to_do_list.difference(done_list))
to_do_list

['xg',
 'qw',
 'ur',
 'gc',
 'lq',
 'vq',
 'pz',
 'yq',
 'vp',
 'zk',
 'zq',
 'kz',
 'xf',
 'jq',
 'zt',
 'zf',
 'rc',
 'pt',
 'xq',
 'zj',
 'uq',
 'bq',
 'gz',
 'xz',
 'qj',
 'gq',
 'qz',
 'xb',
 'kq',
 'gn',
 'zg',
 'fq',
 'qh',
 'yj',
 'zz',
 'rq',
 'wz',
 'qm',
 'wq',
 'yz',
 'pk',
 'rz',
 'qf',
 'xw',
 'jz',
 'qg']

In [5]:
def delist_lang(lst):
    lang_lst=[]
    for i,lang in enumerate(lst):
        if not lang:
            lang_lst.append(None)
        else:
            lang_lst.append(lang[0])
    return lang_lst


def significance(lst):
    significance_list=[]
    for l in lst:
        if len(l)>1:
            significance_list.append(abs(l[0]-l[1])/np.mean(l[0]+l[1])>0.1)
            #print(f'{conf[0]} {conf[1]} {abs(conf[0]-conf[1])/np.mean(conf[0]+conf[1])>0.1}')
        else:
            significance_list.append(True)
    return significance_list

In [6]:
def sent_maker(sent_lst):
    ret_sents=[]
    g_pos=[]
    for sent in sent_lst:
        cur_words=[]
        pos_sent=[]
        sent=sent.replace('_END_','@@@_.')
        for word_pos in sent.split(' '):
            word,pos=word_pos.rsplit('_',1)
            cur_words.append(word)
            pos_sent.append(pos)
            cur_sent=' '.join(cur_words)
            cur_pos=' '.join(pos_sent)
        ret_sents.append(cur_sent)
        g_pos.append(cur_pos)
    return ret_sents,g_pos

In [7]:
def ner_lemma_reducer(sent):
    ner_sent=[]
    lemma=[]
    pos=[]
    #parse=[]
    is_comp=False
    ner_token=[]
    ner_length=[]
    ner=[]
    parsed_sent=nlp(sent)
    for token in parsed_sent:
        #parse.append(token.text)
        lemma.append(token.lemma_)
        pos.append(token.pos_)
        if token.ent_type_=="":
            to_add="NONNER"
        else:
            to_add=token.ent_type_
        ner_token.append(to_add)
        if token.dep_=="compound":
            is_comp=True
    #print(parse)
    #parse_sent=' '.join(parse)
    lemma_sent=' '.join(lemma)
    pos_sent=' '.join(pos)
    ner_token_sent=' '.join(ner_token)
    #dep_sent=' '.join(dep)
    ner_length=0
    if parsed_sent.ents:
        for ent in parsed_sent.ents:
            #cur_ner=
            #cur_ner='_'.join([str(ent.start_char), str(ent.end_char), ent.label_])
            ner_length+=ent.end_char-ent.start_char
            #ner.append(cur_ner)
    #else:
        #ner.append("")
    ner_sent=' '.join(ner)
    
    return ner_token_sent,ner_length,lemma_sent,pos_sent,is_comp

In [8]:
def lang_tagger(parsed_sent):
    labels,confs=fmodel.predict(parsed_sent,k=-1,threshold=0.1)
    lang_list=delist_lang(labels)    
    significance_list=significance(confs)
    assert len(lang_list)==len(significance_list)
    return lang_list,significance_list

In [9]:
def str_joiner(df):
    #print(df)
    new_df=pd.DataFrame()
    try:
        new_df[['l1','l2','l3','l4','l5']]=df.lemma_sent.str.split(" ",expand=True)
        new_df[['p1','p2','p3','p4','p5']]=df.pos_sent.str.split(" ",expand=True)
    except:
        print(df)
    new_df['lemma_pos']=new_df.l1+"_"+new_df.p1+" "+\
                        new_df.l2+"_"+new_df.p2+" "+\
                        new_df.l3+"_"+new_df.p3+" "+\
                        new_df.l4+"_"+new_df.p4+" "+\
                        new_df.l5+"_"+new_df.p5
    return new_df['lemma_pos']

In [10]:
def trial(df):
    df.reset_index(inplace=True,drop=True)
    ret_lst=sent_maker(df.old_index)
    
    df['sent']=ret_lst[0]
    df['g_pos']=ret_lst[1]
    
    results=np.vectorize(ner_lemma_reducer)(df.sent.values)
    results_df=pd.DataFrame(results)
    results_df=results_df.transpose()
    #results_df.columns=ner_token_sent,ner_length,lemma_sent,pos_sent,is_comp
    results_df.columns=['ner_token_sent','ner_length','lemma_sent','pos_sent','is_comp']


    index_df=pd.concat([df,results_df],axis=1,ignore_index=False)

    lang_list,significance_list=lang_tagger(index_df.sent.values.tolist())
    index_df['lang']=lang_list
    index_df['lang_conf']=significance_list
    index_df.lang=index_df.lang.str.split('_',n=4).str[-1]
    index_df=index_df.loc[(index_df.lang=='en') &(index_df.lang_conf==True)]

    index_df['nwords']=index_df.pos_sent.str.count(' ').add(1)
    index_df=index_df.loc[index_df.nwords==5]
    
    index_df.lemma_sent=index_df.lemma_sent.str.lower()
    #index_df.pos_sent=index_df.pos_sent.str.replace('PROPN','NOUN',regex=False)
    #index_df.pos_sent=index_df.pos_sent.str.replace('AUX','VERB',regex=False)
    #index_df.pos_sent=index_df.pos_sent.str.replace('CCONJ','CONJ',regex=False)
    #index_df.g_pos=index_df.g_pos.str.replace('.','PUNCT',regex=False)
    #index_df.g_pos=index_df.g_pos.str.replace('PRT','ADP',regex=False)
    if index_df.shape[0]==0:
        return index_df
    index_df['lemma_pos']=str_joiner(index_df)
    index_df['nX']=index_df.pos_sent.str.count('X')-index_df.pos_sent.str.count('AUX')
    index_df=index_df.loc[~(index_df.nX>1)]
    
    index_df['ner_perc']=index_df.ner_length/index_df.sent.str.len()
   
    index_df['comp_class']=0

    index_df.loc[index_df.pos_sent.str.contains(n1),'comp_class']=1
    index_df.loc[~(index_df.pos_sent.str.contains(n1))& index_df.pos_sent.str.contains(n2),'comp_class']=2
    index_df.loc[index_df.pos_sent.str.contains(n3),'comp_class']=3
    index_df.loc[index_df.pos_sent.str.contains(n4),'comp_class']=4
    index_df.loc[~(index_df.pos_sent.str.contains(n1))& index_df.pos_sent.str.contains(n5),'comp_class']=5
    index_df.drop(['lang','lang_conf','nwords','nX','lemma_sent','ner_length'],axis=1,inplace=True)
    #print(index_df)
    return index_df

In [11]:
def large_df_processor(letter,num_partitions):
    
    #CHUNKSIZE = 1_000_000_000
    CHUNKSIZE=10_000_000
    total_df_shape=0
    df_list=[]
    path_loc="http://storage.googleapis.com/books/ngrams/books/googlebooks-eng-all-5gram-20120701-"+letter+".gz"
    dfs   = pd.read_csv(path_loc, compression='gzip', header=None, sep="\t", quoting=csv.QUOTE_NONE,usecols=[0,1,2],chunksize=CHUNKSIZE)    
    for i,df in enumerate(dfs):


        print(f'Split num {i+1}')        
        cur_time=time.time()
        df.columns=['fivegram_pos','year','count']
        #df=df.loc[df.year>=1800]
        index_df=df.groupby(['fivegram_pos'])['count'].sum().reset_index()
        index_df.columns=['old_index','total_count']
        index_df=index_df.loc[index_df.old_index.str.match("^"+keep_string*5+"$",na=False)]

        df_split = np.array_split(index_df, num_partitions)
        pool = Pool(num_partitions)
        print('Started parallelization')
        results=pool.map_async(trial,df_split)
        pool.close()
        pool.join()
        
        
        curr_df_list=results.get()
        new_index_df=pd.concat(curr_df_list,ignore_index=True)
        
        
        print(f'Total time taken for split num {0+1}: {round(time.time()-cur_time)} secs')        
    
        ntypes=new_index_df.shape[0]
        ntokens=new_index_df.total_count.sum()

        types_perc=round(ntypes/df.shape[0]*100,3)
        print(f'Number of types: {ntypes}, perc. of unique types (decade agnostic): {types_perc}%')

        print(f'Number of tokens: {ntokens}, ratio of tokens to types: {round(ntokens/ntypes,3)}')

        ncomptypes=np.sum(new_index_df.comp_class!=0)
        ncomptypes_perc=round(ncomptypes/ntypes*100,3)
        print(f'Number of compounds types: {ncomptypes}, perc. of compound types: {ncomptypes_perc}%')

        comp_count=new_index_df.loc[new_index_df.comp_class!=0,'total_count'].sum()
        comp_count_perc=round(comp_count/ntokens*100,3)
        print(f'Compound count: {comp_count}, perc. of compound tokens: {comp_count_perc}%')

        words_df=new_index_df.loc[new_index_df.pos_sent.str.contains('(?:NOUN|PROPN)')].reset_index(drop=True)
        #words_df['nner']=words_df.ner_sent.str.count(' ').add(1)
        words_df.comp_class=words_df.comp_class.astype('int32')
        return words_df
        words=pd.merge(df,words_df,left_on='fivegram_pos',right_on='old_index',how='right')
        words=words.groupby(['lemma_pos','pos_sent','year','comp_class','ner_token_sent','ner_perc','is_comp'])['count'].sum().to_frame()
        words.reset_index(inplace=True)

        words.to_pickle(f'/data/dharp/compounds/datasets/google/{letter}{i+1}.pkl')
        #phrases_df=words_df.loc[words_df.pos_sent.str.contains('NOUN NOUN')].reset_index(drop=True)
        #phrases=pd.merge(df,phrases_df,left_on='fivegram_pos',right_on='old_index',how='right')
        #phrases=phrases.groupby(['lemma_sent','year','pos_sent','comp_class','ner_sent'])['count'].sum().to_frame()
        #phrases.reset_index(inplace=True)

        #comp_df=phrases_df.loc[phrases_df.comp_class!=0].reset_index(drop=True)
        #compounds=pd.merge(df,comp_df,left_on='fivegram_pos',right_on='old_index',how='right')
        #compounds=compounds.groupby(['lemma_sent','year','pos_sent','comp_class','ner_sent'])['count'].sum().to_frame()
        #compounds.reset_index(inplace=True)

        print(f'Total time taken for letter {letter}: {round(time.time()-cur_time)} secs')
        with open(f'/data/dharp/compounds/datasets/stats/{letter}{i+1}.txt','w') as f:
            f.write(f'{letter}\t{i+1}\t{ntypes}\t{ntokens}\t{ncomptypes}\t{comp_count}\n')

In [12]:
num_partitions=round(0.9*mp.cpu_count())
cur_df=large_df_processor('co',num_partitions)

Split num 1
Started parallelization
Total time taken for split num 1: 63 secs
Number of types: 122472, perc. of unique types (decade agnostic): 1.225%
Number of tokens: 18561214, ratio of tokens to types: 151.555
Number of compounds types: 9413, perc. of compound types: 7.686%
Compound count: 1332399, perc. of compound tokens: 7.178%


In [16]:
cur_df.loc[(cur_df.is_comp==True )& (cur_df.lemma_pos.str.contains('coffee'))].sample(30)

,old_index,total_count,sent,g_pos,ner_token_sent,pos_sent,is_comp,lemma_pos,ner_perc,comp_class
15889,coffee_NOUN at_ADP the_DET student_NOUN union_...,96,coffee at the student union,NOUN ADP DET NOUN NOUN,NONNER NONNER NONNER NONNER NONNER,NOUN ADP DET NOUN NOUN,True,coffee_NOUN at_ADP the_DET student_NOUN union_...,0.0,5
825,"Coffee_NOUN ,_. with_ADP nine_NUM hundred_NUM",61,"Coffee , with nine hundred",NOUN . ADP NUM NUM,NONNER NONNER NONNER CARDINAL CARDINAL,NOUN PUNCT ADP NUM NUM,True,"coffee_NOUN ,_PUNCT with_ADP nine_NUM hundred_NUM",0.461538,0
15786,"coffee_ADJ -_. tree_NOUN ,_. which_DET",115,"coffee - tree , which",ADJ . NOUN . DET,NONNER NONNER NONNER NONNER NONNER,NOUN PUNCT NOUN PUNCT DET,True,"coffee_NOUN -_PUNCT tree_NOUN ,_PUNCT which_DET",0.0,0
15891,coffee_NOUN breaks_NOUN and_CONJ lunch_NOUN pe...,92,coffee breaks and lunch periods,NOUN NOUN CONJ NOUN NOUN,NONNER NONNER NONNER NONNER NONNER,NOUN NOUN CCONJ NOUN NOUN,True,coffee_NOUN break_NOUN and_CCONJ lunch_NOUN pe...,0.0,1
15794,"coffee_NOUN ,_. and_CONJ fruit_NOUN juices_NOUN",76,"coffee , and fruit juices",NOUN . CONJ NOUN NOUN,NONNER NONNER NONNER NONNER NONNER,NOUN PUNCT CCONJ NOUN NOUN,True,"coffee_NOUN ,_PUNCT and_CCONJ fruit_NOUN juice...",0.0,5
15779,coffee_ADJ -_. house_NOUN either_CONJ to_PRT,51,coffee - house either to,ADJ . NOUN CONJ PRT,NONNER NONNER NONNER NONNER NONNER,NOUN PUNCT NOUN ADV ADP,True,coffee_NOUN -_PUNCT house_NOUN either_ADV to_ADP,0.0,0
15936,"coffee_NOUN shop_NOUN ,_. and_CONJ in_ADP",53,"coffee shop , and in",NOUN NOUN . CONJ ADP,NONNER NONNER NONNER NONNER NONNER,NOUN NOUN PUNCT CCONJ ADP,True,"coffee_NOUN shop_NOUN ,_PUNCT and_CCONJ in_ADP",0.0,2
15939,coffee_NOUN shop_NOUN downstairs_NOUN ._. _END_,230,coffee shop downstairs . @@@,NOUN NOUN NOUN . .,NONNER NONNER NONNER NONNER NONNER,NOUN NOUN ADV PUNCT PUNCT,True,coffee_NOUN shop_NOUN downstairs_ADV ._PUNCT @...,0.0,2
15937,coffee_NOUN shop_NOUN a_DET couple_NOUN of_ADP,80,coffee shop a couple of,NOUN NOUN DET NOUN ADP,NONNER NONNER NONNER NONNER NONNER,NOUN NOUN DET NOUN ADP,True,coffee_NOUN shop_NOUN a_DET couple_NOUN of_ADP,0.0,2
15925,"coffee_NOUN mug_NOUN ._. ""_. _END_",124,"coffee mug . "" @@@",NOUN NOUN . . .,NONNER NONNER NONNER NONNER NONNER,NOUN NOUN PUNCT PUNCT PUNCT,True,"coffee_NOUN mug_NOUN ._PUNCT ""_PUNCT @@@_PUNCT",0.0,2


Keep PROP NOUN

Change Dep to Is_Comp

(later) remove compounds based on PROPN, NER

In [19]:
cur_df.is_comp.value_counts(normalize=True)

False    0.907788
True     0.092212
Name: is_comp, dtype: float64

In [21]:
cur_df.comp_class.value_counts(normalize=True)

0    0.921826
2    0.033982
4    0.020835
5    0.017700
1    0.003347
3    0.002310
Name: comp_class, dtype: float64

cur_df.loc[(cur_df.comp_class==0) & (cur_df.is_comp==True)].sample(30)

new_index_df['sent_len']=new_index_df.parse_sent.str.len()
new_index_df['ner_ratio']=new_index_df.ner_length/new_index_df.sent_len
new_index_df.loc[new_index_df.ner_ratio>=0.5]

In [20]:
for letter in to_do_list:
    #num_partitions=250
    path_loc="http://storage.googleapis.com/books/ngrams/books/googlebooks-eng-all-5gram-20120701-"+letter+".gz"
    df   = pd.read_csv(path_loc, compression='gzip', header=None, sep="\t", quoting=csv.QUOTE_NONE,usecols=[0,1,2])    
   
    cur_time=time.time()
    df.columns=['fivegram_pos','year','count']
    #df=df.loc[df.year>=1800]
    index_df=df.groupby(['fivegram_pos'])['count'].sum().reset_index()
    index_df.columns=['old_index','total_count']
    index_df=index_df.loc[index_df.old_index.str.match("^"+keep_string*5+"$",na=False)]

    old_types=index_df.shape[0]
    old_tokens=index_df.total_count.sum()
    #df_split = np.array_split(index_df, num_partitions)
    #pool = Pool(num_partitions)
    #print('Started parallelization')
    #results=pool.map_async(trial,df_split)
    #pool.close()
    #pool.join()
        
        
    #  curr_df_list=results.get()
    #new_index_df=pd.concat(curr_df_list,ignore_index=True)
    new_index_df=trial(index_df)
    new_types=new_index_df.shape[0]
    new_tokens=new_index_df.total_count.sum()

    print(f'Change in number of types: {(new_types-old_types)/old_types*100}')
    print(f'Change in number of tokens: {(new_tokens-old_tokens)/old_tokens*100}')
    print(f'Total time taken for split num {0+1}: {round(time.time()-cur_time)} secs')        
    
    ntypes=new_index_df.shape[0]
    ntokens=new_index_df.total_count.sum()

    types_perc=round(ntypes/df.shape[0]*100,3)
    print(f'Number of types: {ntypes}, perc. of unique types (decade agnostic): {types_perc}%')

    print(f'Number of tokens: {ntokens}, ratio of tokens to types: {round(ntokens/ntypes,3)}')

    ncomptypes=np.sum(new_index_df.comp_class!=0)
    ncomptypes_perc=round(ncomptypes/ntypes*100,3)
    print(f'Number of compounds types: {ncomptypes}, perc. of compound types: {ncomptypes_perc}%')

    comp_count=new_index_df.loc[new_index_df.comp_class!=0,'total_count'].sum()
    comp_count_perc=round(comp_count/ntokens*100,3)
    print(f'Compound count: {comp_count}, perc. of compound tokens: {comp_count_perc}%')

    words_df=new_index_df.loc[new_index_df.pos_sent.str.contains('(?:NOUN|PROPN)')].reset_index(drop=True)
        #words_df['nner']=words_df.ner_sent.str.count(' ').add(1)
    words_df.comp_class=words_df.comp_class.astype('int32')

    words=pd.merge(df,words_df,left_on='fivegram_pos',right_on='old_index',how='right')
    words=words.groupby(['lemma_pos','pos_sent','year','comp_class','ner_token_sent','ner_perc','is_comp'])['count'].sum().to_frame()
    words.reset_index(inplace=True)

    words.to_pickle(f'/data/dharp/compounds/datasets/google/{letter}{0+1}.pkl')
        #phrases_df=words_df.loc[words_df.pos_sent.str.contains('NOUN NOUN')].reset_index(drop=True)
        #phrases=pd.merge(df,phrases_df,left_on='fivegram_pos',right_on='old_index',how='right')
        #phrases=phrases.groupby(['lemma_sent','year','pos_sent','comp_class','ner_sent'])['count'].sum().to_frame()
        #phrases.reset_index(inplace=True)

        #comp_df=phrases_df.loc[phrases_df.comp_class!=0].reset_index(drop=True)
        #compounds=pd.merge(df,comp_df,left_on='fivegram_pos',right_on='old_index',how='right')
        #compounds=compounds.groupby(['lemma_sent','year','pos_sent','comp_class','ner_sent'])['count'].sum().to_frame()
        #compounds.reset_index(inplace=True)

    print(f'Total time taken for letter {letter}: {round(time.time()-cur_time)} secs')
    with open(f'/data/dharp/compounds/datasets/stats/{letter}{0+1}.txt','w') as f:
        f.write(f'{letter}\t{0+1}\t{ntypes}\t{ntokens}\t{ncomptypes}\t{comp_count}\n')


Change in number of types: -100.0
Change in number of tokens: -100.0
Total time taken for split num 1: 0 secs
Number of types: 0, perc. of unique types (decade agnostic): 0.0%
Number of tokens: 0, ratio of tokens to types: nan


/tmp/ipykernel_193837/2514220630.py:39: RuntimeWarning: invalid value encountered in long_scalars
  print(f'Number of tokens: {ntokens}, ratio of tokens to types: {round(ntokens/ntypes,3)}')


AttributeError: 'DataFrame' object has no attribute 'comp_class'

In [ ]:
words.is_comp.value_counts()

In [16]:
words.loc[words.year<1600]

,lemma_sent,year,pos_sent,comp_class,count
196051,babble and loss of time,1581,VERB CCONJ NOUN ADP NOUN,0,1
286771,babe wrap in swaddle clothe,1581,NOUN VERB ADP VERB NOUN,0,2
511228,"baby , the death of",1587,NOUN PUNCT DET NOUN ADP,0,2
1054041,"babylon , and she be",1520,NOUN PUNCT CCONJ PRON AUX,0,1
1073153,"babylon , my servant ;",1582,NOUN PUNCT PRON NOUN PUNCT,0,1
...,...,...,...,...,...
47216424,bay of biscay . @@@,1575,NOUN ADP NOUN PUNCT PUNCT,0,1
47228558,bay of espiritu santo .,1524,NOUN ADP NOUN NOUN PUNCT,4,1
47243376,"bay of honduras , the",1524,NOUN ADP NOUN PUNCT DET,0,1
47382774,"bay tree , and high",1584,NOUN NOUN PUNCT CCONJ ADJ,2,1


In [14]:
words.loc[words.comp_class==3].sample(30)

,lemma_sent,year,pos_sent,comp_class,count
36969442,basic datum source be the,1983,ADJ NOUN NOUN AUX DET,3,3
36894419,basic clothing need . @@@,2000,ADJ NOUN NOUN PUNCT PUNCT,3,7
6548509,back pain care from family,2000,ADJ NOUN NOUN ADP NOUN,3,8
31455144,base budgeting requirements in 47,2007,VERB NOUN NOUN ADP NUM,3,10
26180447,bar iron gate . @@@,2006,VERB NOUN NOUN PUNCT PUNCT,3,1
31235862,base acculturation scale for mexican,2002,VERB NOUN NOUN ADP NOUN,3,2
37306624,basic material need of the,1999,ADJ NOUN NOUN ADP DET,3,7
37282435,basic lead acetate and ammonia,1882,ADJ NOUN NOUN CCONJ NOUN,3,1
36982875,basic design process . @@@,1991,ADJ NOUN NOUN PUNCT PUNCT,3,1
36895540,basic college education . @@@,1968,ADJ NOUN NOUN PUNCT PUNCT,3,1


In [31]:
new_index_df.loc[new_index_df.comp_class!=0].ner_sent.value_counts(normalize=True)*100

                            53.884922
0_15_ORG                     0.445087
0_14_ORG                     0.438419
0_6_PERSON 9_14_PERSON       0.382852
0_21_ORG                     0.375629
                              ...    
0_4_GPE 17_38_ORG            0.000556
30_37_PRODUCT                0.000556
10_14_PERCENT                0.000556
0_14_GPE 17_30_PERSON        0.000556
0_15_PERSON 21_25_PERSON     0.000556
Name: ner_sent, Length: 12226, dtype: float64

In [29]:
new_index_df.loc[(new_index_df.ner_length>1)&(new_index_df.comp_class!=0)]

,old_index,total_count,sent,g_pos,parse_sent,ner_sent,ner_length,lemma_sent,pos_sent,lang,lang_conf,nwords,nX,comp_class
15,"B.A.R._NOUN ,_. 122_NUM Banbury_NOUN Road_NOUN",206,"B.A.R. , 122 Banbury Road",NOUN . NUM NOUN NOUN,"B.A.R. , 122 Banbury Road",9_12_CARDINAL,3,"b.a.r. , 122 banbury road",NOUN PUNCT NUM NOUN NOUN,en,True,5,0,5
60,"B.A.R._NOUN B.A.R._NOUN ,_. 122_NUM Banbury_NOUN",62,"B.A.R. B.A.R. , 122 Banbury",NOUN NOUN . NUM NOUN,"B.A.R. B.A.R. , 122 Banbury",0_13_PERSON 16_19_CARDINAL 20_27_ORG,23,"b.a.r. b.a.r. , 122 banbury",NOUN NOUN PUNCT NUM NOUN,en,True,5,0,2
67,B.A.R_NOUN prices_NOUN are_VERB inclusive_ADJ ...,83,B.A.R prices are inclusive of,NOUN NOUN VERB ADJ ADP,B.A.R prices are inclusive of,0_5_ORG,5,b.a.r price be inclusive of,NOUN NOUN AUX ADJ ADP,en,True,5,0,2
68,B.A.R_NOUN publications_NOUN in_ADP print_NOUN...,86,B.A.R publications in print please,NOUN NOUN ADP NOUN VERB,B.A.R publications in print please,0_5_ORG,5,b.a.r publication in print please,NOUN NOUN ADP NOUN INTJ,en,True,5,0,2
69,B.A._ADJ -_. A_ADJ Dictionary_ADJ and_CONJ,86,B.A. - A Dictionary and,ADJ . ADJ ADJ CONJ,B.A. - A Dictionary and,0_19_ORG,19,b.a. - a dictionary and,NOUN PUNCT NOUN NOUN CCONJ,en,True,5,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1574756,"bazaars_NOUN of_ADP Central_NOUN Asia_NOUN ,_.",62,"bazaars of Central Asia ,",NOUN ADP NOUN NOUN .,"bazaars of Central Asia ,",11_23_LOC,12,"bazaar of central asia ,",NOUN ADP NOUN NOUN PUNCT,en,True,5,0,4
1574757,bazaars_NOUN of_ADP Central_NOUN Asia_NOUN ._.,42,bazaars of Central Asia .,NOUN ADP NOUN NOUN .,bazaars of Central Asia .,11_23_LOC,12,bazaar of central asia .,NOUN ADP NOUN NOUN PUNCT,en,True,5,0,4
1574770,bazaars_NOUN of_ADP the_DET Middle_NOUN East_NOUN,186,bazaars of the Middle East,NOUN ADP DET NOUN NOUN,bazaars of the Middle East,11_26_LOC,15,bazaar of the middle east,NOUN ADP DET NOUN NOUN,en,True,5,0,5
1574771,bazaars_NOUN of_ADP the_DET Near_ADJ East_NOUN,75,bazaars of the Near East,NOUN ADP DET ADJ NOUN,bazaars of the Near East,11_24_LOC,13,bazaar of the near east,NOUN ADP DET NOUN NOUN,en,True,5,0,5


In [28]:
for ner in ner_cats:
    print(ner)
    print(new_index_df.shape[0])
    display(new_index_df.loc[(new_index_df.ner_sent.str.contains(ner))&(new_index_df.comp_class!=0)].sample(30))

CARDINAL
1574921


,old_index,total_count,sent,g_pos,parse_sent,ner_sent,ner_length,lemma_sent,pos_sent,lang,lang_conf,nwords,nX,comp_class
1323704,basic_ADJ four_NUM food_NOUN groups_NOUN and_CONJ,170,basic four food groups and,ADJ NUM NOUN NOUN CONJ,basic four food groups and,6_10_CARDINAL,4,basic four food group and,ADJ NUM NOUN NOUN CCONJ,en,True,5,0,4
1152321,base_NOUN pressure_NOUN of_ADP 2_NUM x_NOUN,133,base pressure of 2 x,NOUN NOUN ADP NUM NOUN,base pressure of 2 x,17_18_CARDINAL,1,base pressure of 2 x,NOUN NOUN ADP NUM SYM,en,True,5,0,2
906152,bank_NOUN account_NOUN is_VERB one_NUM of_ADP,50,bank account is one of,NOUN NOUN VERB NUM ADP,bank account is one of,16_19_CARDINAL,3,bank account be one of,NOUN NOUN AUX NUM ADP,en,True,5,0,2
216685,Battle_NOUN Soldier_NOUN of_ADP Five_NUM Wars_...,71,Battle Soldier of Five Wars,NOUN NOUN ADP NUM NOUN,Battle Soldier of Five Wars,18_22_CARDINAL,4,battle soldier of five war,NOUN NOUN ADP NUM NOUN,en,True,5,0,2
606659,backed_ADJ three_ADJ -_. toed_ADJ woodpecker_NOUN,143,backed three - toed woodpecker,ADJ ADJ . ADJ NOUN,backed three - toed woodpecker,7_12_CARDINAL,5,back three - toed woodpecker,VERB NUM PUNCT NOUN NOUN,en,True,5,0,5
130697,Bar_NOUN =_NUM 50_NUM ym_NOUN ._.,61,Bar = 50 ym .,NOUN NUM NUM NOUN .,Bar = 50 ym .,6_8_CARDINAL,2,bar = 50 ym .,NOUN NOUN NUM INTJ PUNCT,en,True,5,0,2
116381,Bankruptcy_NOUN Act_NOUN (_. 46_NUM &_CONJ,46,Bankruptcy Act ( 46 &,NOUN NOUN . NUM CONJ,Bankruptcy Act ( 46 &,17_19_CARDINAL,2,bankruptcy act ( 46 &,NOUN NOUN PUNCT NUM CCONJ,en,True,5,0,2
872379,band_NOUN width_NOUN of_ADP 10_NUM kc_NOUN,56,band width of 10 kc,NOUN NOUN ADP NUM NOUN,band width of 10 kc,14_16_CARDINAL,2,band width of 10 kc,NOUN NOUN ADP NUM ADV,en,True,5,0,2
1180255,based_VERB on_ADP ODP_NOUN site_NOUN 677_.,61,based on ODP site 677,VERB ADP NOUN NOUN .,based on ODP site 677,9_12_ORG 18_21_CARDINAL,6,base on odp site 677,VERB ADP NOUN NOUN NUM,en,True,5,0,4
842835,ballot_NOUN count_NOUN for_ADP one_NUM of_ADP,95,ballot count for one of,NOUN NOUN ADP NUM ADP,ballot count for one of,17_20_CARDINAL,3,ballot count for one of,NOUN NOUN ADP NUM ADP,en,True,5,0,2


DATE
1574921


,old_index,total_count,sent,g_pos,parse_sent,ner_sent,ner_length,lemma_sent,pos_sent,lang,lang_conf,nwords,nX,comp_class
366716,back_ADV for_ADP the_DET spring_NOUN semester_...,43,back for the spring semester,ADV ADP DET NOUN NOUN,back for the spring semester,9_28_DATE,19,back for the spring semester,ADV ADP DET NOUN NOUN,en,True,5,0,5
795190,balance_NOUN sheets_NOUN of_ADP December_NOUN ...,66,balance sheets of December 31,NOUN NOUN ADP NOUN NUM,balance sheets of December 31,18_29_DATE,11,balance sheet of december 31,NOUN NOUN ADP NOUN NUM,en,True,5,0,2
80810,"Baltimore_NOUN Sun_NOUN ,_. April_NOUN 30_NUM",65,"Baltimore Sun , April 30",NOUN NOUN . NOUN NUM,"Baltimore Sun , April 30",0_13_PERSON 16_24_DATE,21,"baltimore sun , april 30",NOUN NOUN PUNCT NOUN NUM,en,True,5,0,2
148921,Barker_NOUN Lunn_NOUN (_. 1970_NUM )_.,114,Barker Lunn ( 1970 ),NOUN NOUN . NUM .,Barker Lunn ( 1970 ),0_11_PERSON 14_18_DATE,15,barker lunn ( 1970 ),NOUN NOUN PUNCT NUM PUNCT,en,True,5,0,2
647613,backward_ADV into_ADP the_DET Middle_NOUN Ages...,106,backward into the Middle Ages,ADV ADP DET NOUN NOUN,backward into the Middle Ages,14_29_DATE,15,backward into the middle ages,ADV ADP DET NOUN NOUN,en,True,5,0,5
81871,Baltimore_NOUN convention_NOUN of_ADP 1844_NUM...,67,"Baltimore convention of 1844 ,",NOUN NOUN ADP NUM .,"Baltimore convention of 1844 ,",0_9_GPE 24_28_DATE,13,"baltimore convention of 1844 ,",NOUN NOUN ADP NUM PUNCT,en,True,5,0,2
116506,Bankruptcy_NOUN Act_NOUN of_ADP 1898_NUM ._.,1016,Bankruptcy Act of 1898 .,NOUN NOUN ADP NUM .,Bankruptcy Act of 1898 .,18_22_DATE,4,bankruptcy act of 1898 .,NOUN NOUN ADP NUM PUNCT,en,True,5,0,2
80976,"Baltimore_NOUN Sun_NOUN ,_. March_NOUN 21_NUM",82,"Baltimore Sun , March 21",NOUN NOUN . NOUN NUM,"Baltimore Sun , March 21",0_13_PERSON 16_24_DATE,21,"baltimore sun , march 21",NOUN NOUN PUNCT NOUN NUM,en,True,5,0,2
190894,"Basic_ADJ Books_NOUN ,_. 1965_NUM ,_.",565,"Basic Books , 1965 ,",ADJ NOUN . NUM .,"Basic Books , 1965 ,",0_11_ORG 14_18_DATE,15,"basic books , 1965 ,",NOUN NOUN PUNCT NUM PUNCT,en,True,5,0,2
73445,"Ballston_NOUN Spa_NOUN ,_. NY_NOUN 12020_.",48,"Ballston Spa , NY 12020",NOUN NOUN . NOUN .,"Ballston Spa , NY 12020",0_12_GPE 15_17_GPE 18_23_DATE,19,"ballston spa , ny 12020",NOUN NOUN PUNCT NOUN NUM,en,True,5,0,2


EVENT
1574921


...

,old_index,total_count,sent,g_pos,parse_sent,ner_sent,ner_length,lemma_sent,pos_sent,lang,lang_conf,nwords,nX,comp_class
997352,bar_NOUN =_NUM 0.1_NUM mm_NOUN )_.,134,bar = 0.1 mm ),NOUN NUM NUM NOUN .,bar = 0.1 mm ),6_12_QUANTITY,6,bar = 0.1 mm ),NOUN NOUN NUM NUM PUNCT,en,True,5,0,2
831949,ball_NOUN stage_NOUN (_. 238_NUM °_NUM,72,ball stage ( 238 °,NOUN NOUN . NUM NUM,ball stage ( 238 °,13_18_QUANTITY,5,ball stage ( 238 °,NOUN NOUN PUNCT NUM NOUN,en,True,5,0,2
49633,Bake_NOUN at_ADP 350_NUM degrees_NOUN F_NOUN,546,Bake at 350 degrees F,NOUN ADP NUM NOUN NOUN,Bake at 350 degrees F,8_19_QUANTITY,11,bake at 350 degree f,VERB ADP NUM NOUN NOUN,en,True,5,0,5
760714,baking_VERB soda_NOUN 1_NUM teaspoon_NUM cream...,221,baking soda 1 teaspoon cream,VERB NOUN NUM NUM NOUN,baking soda 1 teaspoon cream,12_22_QUANTITY,10,bake soda 1 teaspoon cream,VERB NOUN NUM NOUN NOUN,en,True,5,0,5
1071737,barometric_ADJ pressure_NOUN of_ADP 740_NUM mm...,61,barometric pressure of 740 mm,ADJ NOUN ADP NUM NOUN,barometric pressure of 740 mm,23_29_QUANTITY,6,barometric pressure of 740 mm,NOUN NOUN ADP NUM NOUN,en,True,5,0,2
1071735,barometric_ADJ pressure_NOUN of_ADP 29.92_NUM ...,180,barometric pressure of 29.92 inches,ADJ NOUN ADP NUM NOUN,barometric pressure of 29.92 inches,23_35_QUANTITY,12,barometric pressure of 29.92 inch,NOUN NOUN ADP NUM NOUN,en,True,5,0,2
1159188,baseball_NOUN diamond_NOUN is_VERB 90_NUM feet...,72,baseball diamond is 90 feet,NOUN NOUN VERB NUM NOUN,baseball diamond is 90 feet,20_27_QUANTITY,7,baseball diamond be 90 foot,NOUN NOUN AUX NUM NOUN,en,True,5,0,2
1056759,barium_NOUN flint_VERB 0527_NUM 3_NUM 51.7_NUM,56,barium flint 0527 3 51.7,NOUN VERB NUM NUM NUM,barium flint 0527 3 51.7,13_17_CARDINAL 18_24_QUANTITY,10,barium flint 0527 3 51.7,NOUN NOUN NUM NUM NUM,en,True,5,0,2
1569242,bay_NOUN leaf_NOUN 1_NUM teaspoon_NUM sugar_NOUN,61,bay leaf 1 teaspoon sugar,NOUN NOUN NUM NUM NOUN,bay leaf 1 teaspoon sugar,9_19_QUANTITY,10,bay leaf 1 teaspoon sugar,NOUN NOUN NUM NOUN NOUN,en,True,5,0,1
850717,balsamic_ADJ vinegar_NOUN 3_NUM tablespoons_NO...,41,balsamic vinegar 3 tablespoons olive,ADJ NOUN NUM NOUN ADJ,balsamic vinegar 3 tablespoons olive,17_30_QUANTITY,13,balsamic vinegar 3 tablespoon olive,ADJ NOUN NUM NOUN NOUN,en,True,5,0,5


TIME
1574921


,old_index,total_count,sent,g_pos,parse_sent,ner_sent,ner_length,lemma_sent,pos_sent,lang,lang_conf,nwords,nX,comp_class
1220770,based_VERB on_ADP machine_NOUN hours_NOUN ._.,379,based on machine hours .,VERB ADP NOUN NOUN .,based on machine hours .,9_22_TIME,13,base on machine hour .,VERB ADP NOUN NOUN PUNCT,en,True,5,0,4
55539,Baking_VERB time_NOUN :_NUM 1_NUM hour_NOUN,127,Baking time : 1 hour,VERB NOUN NUM NUM NOUN,Baking time : 1 hour,14_20_TIME,6,baking time : 1 hour,NOUN NOUN PUNCT NUM NOUN,en,True,5,0,2
363227,"back_ADV early_ADJ tomorrow_NOUN morning_NOUN ,_.",95,"back early tomorrow morning ,",ADV ADJ NOUN NOUN .,"back early tomorrow morning ,",5_27_TIME,22,"back early tomorrow morning ,",ADV ADV NOUN NOUN PUNCT,en,True,5,0,4
1154768,base_NOUN wages_NOUN between_ADP night_NOUN an...,45,base wages between night and,NOUN NOUN ADP NOUN CONJ,base wages between night and,11_24_TIME,13,base wage between night and,NOUN NOUN ADP NOUN CCONJ,en,True,5,0,2
1018210,barbecue_NOUN fire_NOUN on_ADP Thursday_NOUN n...,40,barbecue fire on Thursday nights,NOUN NOUN ADP NOUN NOUN,barbecue fire on Thursday nights,17_25_DATE 26_32_TIME,14,barbecue fire on thursday night,NOUN NOUN ADP NOUN NOUN,en,True,5,0,1
469575,back_ADV to_PRT a_DET summer_NOUN evening_NOUN,48,back to a summer evening,ADV PRT DET NOUN NOUN,back to a summer evening,10_24_TIME,14,back to a summer evening,ADV ADP DET NOUN NOUN,en,True,5,0,5
40127,"Bagdad_NOUN ,_. and_CONJ evening_NOUN conversa...",52,"Bagdad , and evening conversations",NOUN . CONJ NOUN NOUN,"Bagdad , and evening conversations",0_6_GPE 13_20_TIME,13,"bagdad , and evening conversation",NOUN PUNCT CCONJ NOUN NOUN,en,True,5,0,5
1323666,basic_ADJ forty_NUM -_. hour_ADJ week_NOUN,91,basic forty - hour week,ADJ NUM . ADJ NOUN,basic forty - hour week,6_18_TIME,12,basic forty - hour week,ADJ NUM PUNCT NOUN NOUN,en,True,5,0,5
443390,"back_ADV that_ADP Sunday_NOUN evening_NOUN ,_.",40,"back that Sunday evening ,",ADV ADP NOUN NOUN .,"back that Sunday evening ,",10_16_DATE 17_24_TIME,13,"back that sunday evening ,",ADV DET NOUN NOUN PUNCT,en,True,5,0,4
424334,back_ADV on_ADP a_DET Sunday_NOUN night_NOUN,43,back on a Sunday night,ADV ADP DET NOUN NOUN,back on a Sunday night,10_16_DATE 17_22_TIME,11,back on a sunday night,ADV ADP DET NOUN NOUN,en,True,5,0,5


WORK_OF_ART
1574921


,old_index,total_count,sent,g_pos,parse_sent,ner_sent,ner_length,lemma_sent,pos_sent,lang,lang_conf,nwords,nX,comp_class
195915,Basics_NOUN Book_NOUN Everything_NOUN ®_NOUN C...,109,Basics Book Everything ® Craps,NOUN NOUN NOUN NOUN NOUN,Basics Book Everything ® Craps,0_30_WORK_OF_ART,30,basic book everything ® craps,NOUN NOUN PRON VERB NOUN,en,True,5,0,2
190442,"Bashevis_NOUN Singer_NOUN ,_. The_DET Family_NOUN",54,"Bashevis Singer , The Family",NOUN NOUN . DET NOUN,"Bashevis Singer , The Family",0_15_PERSON 18_28_WORK_OF_ART,25,"bashevis singer , the family",NOUN NOUN PUNCT DET NOUN,en,True,5,0,2
1434211,basis_NOUN of_ADP the_DET Prayer_NOUN Book_NOUN,58,basis of the Prayer Book,NOUN ADP DET NOUN NOUN,basis of the Prayer Book,9_24_WORK_OF_ART,15,basis of the prayer book,NOUN ADP DET NOUN NOUN,en,True,5,0,5
178060,"Bas_NOUN Prize_NOUN ,_. i875_X ._.",40,"Bas Prize , i875 .",NOUN NOUN . X .,"Bas Prize , i875 .",0_9_WORK_OF_ART 12_16_PERSON,13,"bas prize , i875 .",NOUN NOUN PUNCT X PUNCT,en,True,5,1,2
157321,"Baron_NOUN ,_. ""_. Black_NOUN Powerlessness_NOUN",105,"Baron , "" Black Powerlessness",NOUN . . NOUN NOUN,"Baron , "" Black Powerlessness",10_29_WORK_OF_ART,19,"baron , "" black powerlessness",NOUN PUNCT PUNCT NOUN NOUN,en,True,5,0,5
44630,"Bailey_NOUN ,_. ""_. Capital_NOUN Gains_NOUN",91,"Bailey , "" Capital Gains",NOUN . . NOUN NOUN,"Bailey , "" Capital Gains",0_6_ORG 11_24_WORK_OF_ART,19,"bailey , "" capital gain",NOUN PUNCT PUNCT NOUN NOUN,en,True,5,0,5
1166215,based_ADJ software_NOUN development_NOUN (_. C...,58,based software development ( CBSD,ADJ NOUN NOUN . X,based software development ( CBSD,29_33_WORK_OF_ART,4,base software development ( cbsd,VERB NOUN NOUN PUNCT NOUN,en,True,5,0,3
222295,Battle_NOUN of_ADP the_DET Fallen_NOUN Timbers...,273,Battle of the Fallen Timbers,NOUN ADP DET NOUN NOUN,Battle of the Fallen Timbers,0_28_WORK_OF_ART,28,battle of the fallen timbers,NOUN ADP DET NOUN NOUN,en,True,5,0,5
838245,"ballet_NOUN The_DET Wooden_NOUN Prince_NOUN ,_.",62,"ballet The Wooden Prince ,",NOUN DET NOUN NOUN .,"ballet The Wooden Prince ,",7_24_WORK_OF_ART,17,"ballet the wooden prince ,",NOUN DET NOUN NOUN PUNCT,en,True,5,0,4
883313,"bands_NOUN playing_VERB ""_. Yankee_ADJ Doodle_...",67,"bands playing "" Yankee Doodle",NOUN VERB . ADJ NOUN,"bands playing "" Yankee Doodle",16_29_WORK_OF_ART,13,"band play "" yankee doodle",NOUN VERB PUNCT NOUN NOUN,en,True,5,0,5
